In [1]:
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit


import qiskit
from qiskit import quantum_info
from qiskit.execute_function import execute
from qiskit import BasicAer
import numpy as np
import pickle
import json
import os
from collections import Counter
from sklearn.metrics import mean_squared_error
from typing import Dict, List
import matplotlib.pyplot as plt

from qiskit import IBMQ, QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, QuantumRegister
from qiskit.qasm import pi
from qiskit.tools.visualization import plot_histogram, circuit_drawer
from qiskit import execute, Aer, BasicAer
import random
from qiskit.circuit.library.standard_gates import RYGate, RYYGate
import cv2 as cv
import pandas as pd

In [2]:
#define utility functions

def simulate(circuit: qiskit.QuantumCircuit) -> dict:
    """Simulate the circuit, give the state vector as the result."""
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circuit, backend)
    result = job.result()
    state_vector = result.get_statevector()
    
    histogram = dict()
    for i in range(len(state_vector)):
        population = abs(state_vector[i]) ** 2
        if population > 1e-9:
            histogram[i] = population
    
    return histogram


def histogram_to_category(histogram):
    """This function take a histogram representations of circuit execution results, and process into labels as described in 
    the problem description."""
    assert abs(sum(histogram.values())-1)<1e-8
    positive=0
    for key in histogram.keys():
        digits = bin(int(key))[2:].zfill(20)
        if digits[-1]=='0':
            positive+=histogram[key]
        
    return positive


def count_gates(circuit: qiskit.QuantumCircuit) -> Dict[int, int]:
    """Returns the number of gate operations with each number of qubits."""
    counter = Counter([len(gate[1]) for gate in circuit.data])
    #feel free to comment out the following two lines. But make sure you don't have k-qubit gates in your circuit
    #for k>2
    for i in range(2,20):
        assert counter[i]==0
        
    return counter


def image_mse(image1,image2):
    # Using sklearns mean squared error:
    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
    return mean_squared_error(image1, image2)

In [3]:
#load the actual hackthon data (fashion-mnist)
# images=np.load(root_dir+'/data/images.npy')
# labels=np.load(root_dir+'/data/labels.npy')

images=np.load('images.npy')
labels=np.load('labels.npy')

#you can visualize it
# !pip install skimage
# from skimage import color
# from skimage import io

# data = np.frombuffer(images[1100], dtype=np.uint8)


In [4]:
def resize(image,size=8):
    resized_image = cv.resize(image,(size,size),interpolation = cv.INTER_NEAREST)
    return resized_image


In [5]:
images_resized = []
for i in images:
    images_resized.append(resize(i))
    
images_resized = np.array(images_resized)

In [6]:
def encoder(image):
# image: a list of 8-bit strings, representing the grey scale intensities of an 8x8 square of pixels
    size = 8
    
    # Initialize the quantum circuit for the image 
    # Pixel position
    idx = QuantumRegister(size, 'idx')
    # Greyscale pixel intensity value
    intensity = QuantumRegister(8,'intensity')
    # Classical register
    cr = ClassicalRegister(size+8, 'cr')

    # Create the quantum circuit for the image
    qc_image = QuantumCircuit(intensity, idx, cr)

    # Set the total number of qubits
    num_qubits = qc_image.num_qubits
    
    # Initialize the quantum circuit

    # Optional: Add Identity gates to the intensity values
    for idx in range(intensity.size):
        qc_image.i(idx)

    # Add Hadamard gates to the pixel positions
    for i in range(0,size):
        qc_image.h(8+i)

        
    ###### Encoding #######
    
    binary = {}
    encoding_size = 8
    for i in range(2**8):
        binary[i] = format(i, '0{}b'.format(encoding_size))
    
    
    for k in range(2 ** encoding_size): 
        # Encode the pixel
        value = image[k]
        # Add the NOT gate to set the position
        position = binary[k]
        for j in range(encoding_size):
            if position[j] == '1':
                qc_image.x(8+j)
        for idx, px_value in enumerate(value[::-1]):
            if(px_value=='1'):
                qc_image.ccx(num_qubits-1, num_qubits-2, idx)
        # Reset the NOT gate
        for j in range(encoding_size):
            if position[j] == '1':
                qc_image.x(8+j)
        
    return qc_image